In [1]:
import os
import csv
import numpy as np
import pandas as pd
import mindspore as ms
from mindspore import nn
from mindspore import context
from mindspore import dataset
from mindspore.train.callback import LossMonitor
from mindspore.common.api import ms_function
from mindspore.ops import operations as P
#当前实验选择算力为Ascend，如果在本地体验，参数device_target设置为"CPU”

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")
import warnings
warnings.filterwarnings("ignore")

In [2]:
#数据预处理
with open('数据/winequality-red.csv') as csv_file:
    data_red = list(csv.reader(csv_file, delimiter=';'))

with open('数据/winequality-white.csv') as csv_file:
    data_white = list(csv.reader(csv_file, delimiter=';'))
#print(np.array(data_red).shape)
#print(np.array(data_white).shape)
# print(data_white)
data = data_red[1:1601]+data_white[1:4900]
list_temp = [0]
list_temp1 = [1]
list_red = []
list_white = []
for i in range(1599):
    list_red.append(list_temp)
for i in range(4898):
    list_white.append(list_temp1)
# print(list_red)
list_y = list_red+list_white
# print(np.array(list_y).shape)
X = np.array([[float(x) for x in s[0:12]] for s in data], np.float32)
Y = np.array([[float(x) for x in s] for s in list_y], np.float32)


# 划分数据集
np.random.seed(42)
train_idx = np.random.choice(6497, 5197, replace=False)
test_idx = np.array(list(set(range(100)) - set(train_idx)))
X_train, Y_train = X[train_idx], Y[train_idx]
X_test, Y_test = X[test_idx], Y[test_idx]
# print(X_train.shape)
#print(X_train.shape)

XY_train = list(zip(X_train, Y_train))
ds_train = dataset.GeneratorDataset(XY_train, ['x', 'y'])
# ds_train.set_dataset_size(80)
ds_train = ds_train.shuffle(buffer_size=5197).batch(16, drop_remainder=True)
#print(ds_train.get_dataset_size())

In [3]:
# 利用mindspore定义Loss
# 使用了交叉损失熵loss
class Loss(nn.Cell):
    def __init__(self):
        super(Loss, self).__init__()
        self.sigmoid_cross_entropy_with_logits = P.SigmoidCrossEntropyWithLogits()
        self.reduce_mean = P.ReduceMean(keep_dims=False)
    def construct(self, x, y):
        loss = self.sigmoid_cross_entropy_with_logits(x, y)
        return self.reduce_mean(loss, -1)

net = nn.Dense(12,1)
loss = Loss()
opt = nn.optim.SGD(net.trainable_params(), learning_rate=0.003)

In [4]:
#利用mindspore模型进行回归训练
model = ms.train.Model(net, loss, opt)
model.train(100, ds_train, callbacks=[LossMonitor(per_print_times=ds_train.get_dataset_size())], dataset_sink_mode=False)

epoch: 1 step: 324, loss is 2.9772164821624756
epoch: 2 step: 324, loss is 16.998912811279297
epoch: 3 step: 324, loss is 5.299436338646046e-07
epoch: 4 step: 324, loss is 10.440601348876953
epoch: 5 step: 324, loss is 1.4120190598987392e-06
epoch: 6 step: 324, loss is 3.3700056076049805
epoch: 7 step: 324, loss is 4.5546345710754395
epoch: 8 step: 324, loss is 6.7940813395963705e-15
epoch: 9 step: 324, loss is 5.390320777893066
epoch: 10 step: 324, loss is 1.6647645235061646
epoch: 11 step: 324, loss is 0.23128409683704376
epoch: 12 step: 324, loss is 17.18328285217285
epoch: 13 step: 324, loss is 11.093755722045898
epoch: 14 step: 324, loss is 3.531242410027141e-13
epoch: 15 step: 324, loss is 2.732475757598877
epoch: 16 step: 324, loss is 5.093698024749756
epoch: 17 step: 324, loss is 1.6259469985961914
epoch: 18 step: 324, loss is 3.0547670648454e-14
epoch: 19 step: 324, loss is 3.6726722717285156
epoch: 20 step: 324, loss is 2.1329846382141113
epoch: 21 step: 324, loss is 1.353304

In [5]:
x = model.predict(ms.Tensor(X_test)).asnumpy()
pred = np.round(1 / (1 + np.exp(-x)))
correct = np.equal(pred, Y_test)
acc = np.mean(correct)
print('Test accuracy by mindspore is', acc)

Test accuracy by mindspore is 1.0
